In [1]:
import ete_lib
import rst_lib

In [4]:
paths, files = rst_lib.build_file_map()

annotation_pairs = [
    rst_lib.build_annotation_pair(files, paths, identifier)
    for identifier in files[rst_lib.TRAIN][rst_lib.DOUBLE]
]

def get_span_scores(spans_1, spans_2, swap=False):
  """
  
  Swap wouldn't matter if we were macro averaging over documents.
  
  How to do micro averaging??
  """
  if swap:
    set_1, set_2 = set_2, set_1
  set1 = set(spans_1.keys())
  set2 = set(spans_2.keys())
  p = len(set1.intersection(set2))/len(set1)
  r = len(set1.intersection(set2))/len(set2)
  f = 2 * p * r / (p + r)
  return f

In [5]:
def get_final_edus(span_map_1, span_map_2):
    final_edu_ends = sorted(
        set(end for _, end in set(span_map_1.keys()).union(span_map_2.keys()))
    )
    final_edu_starts = [0] + final_edu_ends[:-1]
    return list(zip(final_edu_starts, final_edu_ends))

precisions = []
recalls = []
for p in sorted(annotation_pairs, key=lambda x:len(x.input_text)):
    print("=" * 80, p.identifier, len(p.input_text), "=" * 80)
    if p.main_annotation.is_valid and p.double_annotation.is_valid:
        annotations = {
            "main": {"annotation": p.main_annotation},
            "double": {"annotation": p.double_annotation},
        }
        for which, annotation_map in annotations.items():
            annotations[which]["span_map"] = annotation_map["annotation"].get_span_map()
        final_edus = get_final_edus(
            annotations["main"]["span_map"], annotations["double"]["span_map"]
        )
        
        for which, annotation_map in annotations.items():
            annotations[which]["final_span_map"] = annotation_map[
                "annotation"
            ].get_span_map(final_edus)
        

        for which_annotation, annotation_map in annotations.items():
            ete_lib.render(
                annotation_map["annotation"],
                f"binary_parses/{p.identifier}_{which_annotation}.pdf",
            )
            
            
            
            
        print(get_span_scores(annotations["main"]["final_span_map"],annotations["double"]["final_span_map"]))

================================================================================ wsj_1114 179 ================================================================================
1.0
================================================================================ wsj_1100 199 ================================================================================
1.0
================================================================================ wsj_1132 210 ================================================================================
1.0
================================================================================ wsj_1117 244 ================================================================================
1.0
================================================================================ wsj_1153 536 ================================================================================
0.8947368421052632
================================================================================ wsj_1108 

0.9662921348314607
================================================================================ wsj_1314 6702 ================================================================================
0.9323843416370107
================================================================================ wsj_1120 7093 ================================================================================
0.8315018315018317
================================================================================ wsj_1322 9547 ================================================================================
0.8991097922848664
================================================================================ wsj_0633 9562 ================================================================================
0.8917647058823529
================================================================================ wsj_1394 11345 ================================================================================
0.8523908523908523


In [ ]:
sum(scores)/len(scores)